In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Import packets and libraries
import os
import numpy as np
import cv2
from glob import glob
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Activation, BatchNormalization, LeakyReLU
from tensorflow.keras.layers import UpSampling2D, Input, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.applications import MobileNetV2,ResNet50

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.metrics import Recall, Precision
from tensorflow.keras import backend as K

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
IMAGE_SIZE = 256
EPOCHS = 50
BATCH = 10
LR = 1e-4

PATH = "/content/drive/MyDrive//OTU-2D-Dataset-main/OTU-2D-Dataset-main/dataset_split"

In [ ]:
import os
from glob import glob

def load_data(path):
    def load_split(split):
        images = sorted(glob(os.path.join(path, split, 'images', '*')))
        masks = sorted(glob(os.path.join(path, split, 'masks', '*')))
        return images, masks

    train_x, train_y = load_split('train')
    val_x, val_y = load_split('validation')
    test_x, test_y = load_split('test')

    return (train_x, train_y), (val_x, val_y), (test_x, test_y)


In [ ]:
def read_image(path):
    if isinstance(path, bytes):
        path = path.decode('utf-8')
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    if x is None:
        raise ValueError(f"Could not read image: {path}")
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = cv2.resize(x, (IMAGE_SIZE, IMAGE_SIZE))
    x = x.astype(np.float32) / 255.0
    return x

def read_mask(path):
    if isinstance(path, bytes):
        path = path.decode('utf-8')
    y = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    if y is None:
        raise ValueError(f"Could not read mask: {path}")
    y = cv2.resize(y, (IMAGE_SIZE, IMAGE_SIZE))
    y = y.astype(np.float32) / 255.0
    _, y = cv2.threshold(y, 0, 1, cv2.THRESH_BINARY)
    y = np.expand_dims(y, axis=-1)
    return y

def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([IMAGE_SIZE, IMAGE_SIZE, 3])
    y.set_shape([IMAGE_SIZE, IMAGE_SIZE, 1])
    return x, y

def tf_dataset(x, y, batch=8):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.repeat()
    return dataset

In [ ]:
(train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data(PATH)

print("Training data: ", len(train_y))
print("Validation data: ", len(valid_x))
print("Testing data: ", len(test_x))

In [ ]:
import matplotlib.pyplot as plt
import random

def show_random_image_with_mask(x_paths, y_paths, index=None):
    if index is None:
        index = random.randint(0, len(x_paths) - 1)

    image = read_image(x_paths[index])
    mask = read_mask(y_paths[index])

    plt.figure(figsize=(10, 5))

    plt.subplot(1, 2, 1)
    plt.imshow(image)
    plt.title("Ảnh gốc")
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(mask.squeeze(), cmap='gray')
    plt.title("Mask nhị phân")
    plt.axis('off')

    plt.show()


In [ ]:
show_random_image_with_mask(train_x, train_y)


In [ ]:
train_dataset = tf_dataset(train_x, train_y, batch=BATCH)
valid_dataset = tf_dataset(valid_x, valid_y, batch=BATCH)
test_dataset = tf_dataset(test_x, test_y, batch=BATCH)

In [ ]:
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.losses import binary_crossentropy

# Constants
beta = 0.25
alpha = 0.26
gamma = 2.3
epsilon = 1e-7
smooth = 1e-15

# Dice Coefficient
def dice_coef(y_true, y_pred):
    y_true_f = tf.reshape(y_true, [-1])
    y_pred_f = tf.reshape(y_pred, [-1])
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    return (2. * intersection + epsilon) / (
            tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + epsilon)

# Sensitivity
def sensitivity(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    true_positives = tf.reduce_sum(tf.round(tf.clip_by_value(y_true * y_pred, 0, 1)))
    possible_positives = tf.reduce_sum(tf.round(tf.clip_by_value(y_true, 0, 1)))
    return true_positives / (possible_positives + epsilon)

# Specificity
def specificity(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    true_negatives = tf.reduce_sum(tf.round(tf.clip_by_value((1 - y_true) * (1 - y_pred), 0, 1)))
    possible_negatives = tf.reduce_sum(tf.round(tf.clip_by_value(1 - y_true, 0, 1)))
    return true_negatives / (possible_negatives + epsilon)

# Convert probabilities to logits
def convert_to_logits(y_pred):
    y_pred = tf.clip_by_value(y_pred, K.epsilon(), 1 - K.epsilon())
    return tf.math.log(y_pred / (1 - y_pred))

# Weighted cross entropy
def weighted_cross_entropyloss(y_true, y_pred):
    y_pred = convert_to_logits(y_pred)
    pos_weight = beta / (1 - beta)
    loss = tf.nn.weighted_cross_entropy_with_logits(logits=y_pred, labels=y_true, pos_weight=pos_weight)
    return tf.reduce_mean(loss)

# Dice Loss
def generalized_dice_coefficient(y_true, y_pred):
    y_true_f = tf.reshape(y_true, [-1])
    y_pred_f = tf.reshape(y_pred, [-1])
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth)

def dice_loss(y_true, y_pred):
    return 1 - generalized_dice_coefficient(y_true, y_pred)

# Confusion metrics
def confusion(y_true, y_pred):
    y_pred = tf.cast(tf.clip_by_value(y_pred, 0, 1), tf.float32)
    y_true = tf.cast(tf.clip_by_value(y_true, 0, 1), tf.float32)
    y_pred_neg = 1 - y_pred
    y_neg = 1 - y_true
    tp = tf.reduce_sum(y_true * y_pred)
    fp = tf.reduce_sum(y_neg * y_pred)
    fn = tf.reduce_sum(y_true * y_pred_neg)
    precision = (tp + smooth) / (tp + fp + smooth)
    recall = (tp + smooth) / (tp + fn + smooth)
    return precision, recall

# True positive
def true_positive(y_true, y_pred):
    y_pred = tf.round(tf.clip_by_value(y_pred, 0, 1))
    y_true = tf.round(tf.clip_by_value(y_true, 0, 1))
    return (tf.reduce_sum(y_true * y_pred) + smooth) / (tf.reduce_sum(y_true) + smooth)

# True negative
def true_negative(y_true, y_pred):
    y_pred = tf.round(tf.clip_by_value(y_pred, 0, 1))
    y_true = tf.round(tf.clip_by_value(y_true, 0, 1))
    y_pred_neg = 1 - y_pred
    y_neg = 1 - y_true
    return (tf.reduce_sum(y_neg * y_pred_neg) + smooth) / (tf.reduce_sum(y_neg) + smooth)

# Jaccard
def jaccard_similarity(y_true, y_pred):
    y_true_f = tf.reshape(y_true, [-1])
    y_pred_f = tf.reshape(y_pred, [-1])
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    union = tf.reduce_sum(y_true_f + y_pred_f - y_true_f * y_pred_f)
    return intersection / (union + epsilon)

def jaccard_loss(y_true, y_pred):
    return 1 - jaccard_similarity(y_true, y_pred)

# Binary Dice
def binary_dice(y_true, y_pred):
    return 0.5 * binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)



In [ ]:
import tensorflow as tf
import os
import random
import numpy as np
import cv2
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
from keras.utils import normalize
from tqdm import tqdm

from skimage.io import imread, imshow
from skimage.transform import resize
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers as L
from tensorflow.keras.models import Model

# Convolutional block
def conv_block(x, num_filters):
    x = L.Conv2D(num_filters, 3, padding="same", kernel_initializer='he_normal', activation="relu")(x)
    x = L.Conv2D(num_filters, 3, padding="same", kernel_initializer='he_normal', activation="relu")(x)
    return x

# Encoder block
def encoder_block(x, num_filters):
    x = conv_block(x, num_filters)
    p = L.MaxPool2D((2, 2))(x)
    return x, p

# Decoder block
def decoder_block(x, s, num_filters):
    x = L.Conv2DTranspose(num_filters, (2, 2), strides=(2, 2), padding='same')(x)
    x = L.Concatenate()([x, s])
    x = conv_block(x, num_filters)
    return x

# Build U-Net model
def build_unet(input_shape):
    inputs = L.Input(input_shape)

    c1, p1 = encoder_block(inputs, 64)
    c2, p2 = encoder_block(p1, 128)
    c3, p3 = encoder_block(p2, 256)
    c4, p4 = encoder_block(p3, 512)

    c5 = conv_block(p4, 1024)

    d6 = decoder_block(c5, c4, 512)
    d7 = decoder_block(d6, c3, 256)
    d8 = decoder_block(d7, c2, 128)
    d9 = decoder_block(d8, c1, 64)

    outputs = L.Conv2D(1, 1, padding="same", activation="sigmoid")(d9)

    model = Model(inputs, outputs)
    return model


In [ ]:
 input = (256, 256, 3)
model = build_unet(input)
model.summary()

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint_path = "unet_with_original_images.h5"

checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    save_weights_only=False,
    mode='auto',
    save_freq='epoch'
)

callbacks = [
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4),
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=False),
    checkpoint_callback
]

In [ ]:
train_steps = len(train_x) // BATCH
valid_steps = len(valid_x) // BATCH

if len(train_x) % BATCH != 0:
    train_steps += 1
if len(valid_x) % BATCH != 0:
    valid_steps += 1

print(train_steps)
print(valid_steps)

In [ ]:

opt = tf.keras.optimizers.Nadam(LR,clipnorm=1.0)
metrics = [dice_coef, jaccard_similarity, tf.keras.metrics.Recall(), tf.keras.metrics.Precision()]

model.compile(loss=dice_loss, optimizer=opt, metrics=metrics)

In [ ]:
history = model.fit(
    train_dataset,
    validation_data=valid_dataset,
    epochs=50,
    steps_per_epoch=train_steps,
    validation_steps=valid_steps,
    callbacks=callbacks
)


In [ ]:
model.save_weights("/content/drive/MyDrive/unet_with_original_images_weights.weights.h5")


In [ ]:
test_dataset = tf_dataset(test_x, test_y, batch=BATCH)

test_steps = (len(test_x)//BATCH)
if len(test_x) % BATCH != 0:
    test_steps += 1
model.evaluate(test_dataset, steps=test_steps)

In [ ]:
import numpy as np
import cv2
import os
from tqdm import tqdm

def predict_and_save(model, image_path, mask_path, output_dir):
    # Đọc ảnh
    img = read_image(image_path.encode('utf-8'))
    true_mask = read_mask(mask_path.encode('utf-8'))

    img_input = np.expand_dims(img, axis=0)
    pred_mask = model.predict(img_input)[0]


    name = os.path.basename(image_path).split('.')[0]
    out_path = os.path.join(output_dir, name)
    os.makedirs(out_path, exist_ok=True)

    # Lưu ảnh gốc
    img_save = (img * 255).astype(np.uint8)
    img_save = cv2.cvtColor(img_save, cv2.COLOR_RGB2BGR)
    cv2.imwrite(os.path.join(out_path, f"{name}_image.jpg"), img_save)

    # Lưu mask thật
    mask_save = (true_mask * 255).astype(np.uint8)
    cv2.imwrite(os.path.join(out_path, f"{name}_true_mask.png"), mask_save)

    # Lưu mask dự đoán
    pred_mask_save = (pred_mask * 255).astype(np.uint8)
    cv2.imwrite(os.path.join(out_path, f"{name}_pred_mask.png"), pred_mask_save)

def process_predictions(model, data, output_dir):
    x_data, y_data = data
    os.makedirs(output_dir, exist_ok=True)

    for x_path, y_path in tqdm(zip(x_data, y_data), total=len(x_data)):
        predict_and_save(model, x_path, y_path, output_dir)


In [ ]:
output_dir = "/content/drive/MyDrive//OTU-2D-Dataset-main/OTU-2D-Dataset-main/dataset_split/predictions"
process_predictions(model, (test_x, test_y), output_dir)
